In [1]:
!pip install transformers
!pip install datasets
!pip install sentencepiece
!pip install evaluate
!pip install tqdm

!sudo apt-get install git-lfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 38.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [5]:
import torch

from datasets import load_dataset, load_metric, list_metrics
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollator, T5ForConditionalGeneration, T5TokenizerFast
import evaluate
from tqdm import tqdm

from typing import Dict, List, Optional

import dataclasses
from dataclasses import dataclass, field

import logging
import os
import sys

import numpy as np
import torch

from transformers import (
    T5ForConditionalGeneration,
    T5Tokenizer,
    EvalPrediction,
    DataCollator,
    Trainer,
    TrainingArguments)

In [6]:
import pandas as pd

def prep():

    test_data = pd.read_csv('/content/drive/MyDrive/_con.csv')
    test_data.columns = ['Qid','context','questions','answers']
    test_data.to_csv('/content/dataset.csv',encoding='utf-8-sig',index=False)


#prep()

In [8]:
from sklearn.model_selection import train_test_split

# Load the CSV file into a DataFrame
df = pd.read_csv('/content/dataset.csv')
# Split the DataFrame into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Save the train and test DataFrames to new CSV files
train_df.to_csv('train.csv', index=False)
test_df.to_csv('test.csv', index=False)

In [9]:
from datasets import load_dataset

raw_dataset = load_dataset('csv', data_files={'train': "/content/train.csv",
                                              'validation': '/content/test.csv'})



Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [10]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['Qid', 'context', 'questions', 'answers'],
        num_rows: 88285
    })
    validation: Dataset({
        features: ['Qid', 'context', 'questions', 'answers'],
        num_rows: 22072
    })
})

In [7]:
checkpoint ="UBC-NLP/AraT5-base"#"UBC-NLP/AraT5-msa-small" #"t5-base" #AraT5-msa-small
model = T5ForConditionalGeneration.from_pretrained(checkpoint)
tokenizer = T5TokenizerFast.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used

In [8]:
max_input_length =  256
max_target_length = 256

In [9]:
# tokenize the examples
def convert_to_features(example_batch):

    input_encodings = tokenizer.batch_encode_plus(example_batch['context'],
                                                  max_length=max_input_length,
                                                  add_special_tokens=True,
                                                  truncation=True,
                                                  pad_to_max_length=True)

    target_encodings = tokenizer.batch_encode_plus(example_batch['questions'],
                                                   max_length=max_target_length,
                                                   add_special_tokens=True,
                                                   truncation=True, pad_to_max_length=True)

    encodings = {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'decoder_input_ids': target_encodings['input_ids']
        ,'decoder_attention_mask': target_encodings['attention_mask']
    }

    return encodings

def add_eos_examples(example):
  example['context'] = "generate fill-in the blank question: " +example['paragraph'] + " </s>"
  return example



In [20]:
tokenized_dataset  = raw_dataset.map(add_eos_examples)
#tokenized_dataset = tokenized_dataset.map(add_special_tokens)
tokenized_dataset  = tokenized_dataset.map(convert_to_features,  batched=True)

Map:   0%|          | 0/88285 [00:00<?, ? examples/s]

In [22]:
tokenized_dataset["train"][16]["questions"]

"'وفي الحديث: ..... تكافا دماوهم يسعى بذمتهم ادناهم ويرد عليهم اقصاهم اي ابعدهم'"

In [26]:

columns_to_remove = ["context", "questions", "answers"]
train_dataset = tokenized_dataset["train"].remove_columns(columns_to_remove)
valid_dataset = tokenized_dataset["validation"].remove_columns(["answers"])

columns = ['input_ids', 'decoder_input_ids', 'attention_mask', 'decoder_attention_mask']
train_dataset.set_format(type='torch', columns=columns)
valid_dataset.set_format(type='torch', columns=columns)

In [32]:
torch.save(train_dataset, '/content/drive/MyDrive/train_data.pt')
torch.save(valid_dataset, '/content/drive/MyDrive/valid_data.pt')

In [1]:
import torch
train_dataset = torch.load('/content/drive/MyDrive/train_data.pt')
valid_dataset = torch.load('/content/drive/MyDrive/valid_data.pt')

In [2]:
train_dataset

Dataset({
    features: ['Qid', 'input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask'],
    num_rows: 88285
})

In [10]:
# This dataclass implementation is taken from Suraj Patil: https://github.com/patil-suraj/question_generation
@dataclass
class T2TDataCollator():
  def __call__(self, batch: List) -> Dict[str, torch.Tensor]:

    input_ids = torch.stack([example['input_ids'] for example in batch])
    lm_labels = torch.stack([example['decoder_input_ids'] for example in batch])
    lm_labels[lm_labels[:, :] == 0] = -100
    attention_mask = torch.stack([example['attention_mask'] for example in batch])
    decoder_attention_mask = torch.stack([example['decoder_attention_mask'] for example in batch])

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': lm_labels,
        'decoder_attention_mask': decoder_attention_mask
    }

In [11]:
from datasets import load_metric


In [38]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.1 MB/s eta 0:00:00


In [12]:
def compute_metrics(eval_preds):
    #metric = load_metric("bleu")
    sacrebleu = evaluate.load("sacrebleu")
    predictions, labels = eval_preds
    predictions = torch.argmax(predictions.logits,axis=2)
    predictions = predictions.cpu()
    labels = labels.cpu()
    preds =  np.where(predictions != -100, predictions, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    predictions = None
    labels = None
    res = sacrebleu.compute(predictions=decoded_preds,references=decoded_labels)

    return res['score']

def compute_bleu(y_pred, y_true):
    metric = load_metric('bleu')
    metric.add_batch(predictions=y_pred, references=y_true)
    report = metric.compute()
    bleu = report['bleu'] * 100
    return bleu


In [42]:
!pip install accelerate -U
!pip install transformers[torch]

In [13]:
training_args = TrainingArguments(output_dir="/content/drive/MyDrive/Models/T5_Base",
                                  per_device_train_batch_size=4,
                                  per_device_eval_batch_size=4,
                                  gradient_accumulation_steps=8,
                                 # gradient_checkpointing=True,
                                  learning_rate=1e-4,
                                  num_train_epochs=2,
                                  logging_steps=2500,
                                  save_steps = 2500,
                                  evaluation_strategy="steps")
                                  #push_to_hub=True,
                                  #push_to_hub_model_id="t5-end2end-questions-generation")

In [14]:
import gc
gc.collect()

97

In [15]:


# Initialize our Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    #compute_metrics = compute_metrics,
    data_collator=T2TDataCollator()

)

# Training
trainer.train()

# When training is done, we push the fine-tuned model to the Hub
#trainer.push_to_hub("t5-end2end-questions-generation")



Step,Training Loss,Validation Loss
2500,2.232700,0.160375
5000,0.228800,0.129278


TrainOutput(global_step=5518, training_loss=1.1332976205403409, metrics={'train_runtime': 7703.3131, 'train_samples_per_second': 22.921, 'train_steps_per_second': 0.716, 'total_flos': 5.37618473680896e+16, 'train_loss': 1.1332976205403409, 'epoch': 2.0})

In [30]:
trainer.save_model('/content/drive/MyDrive/QG_model')

In [31]:
from transformers import T5ForConditionalGeneration, T5TokenizerFast
#hfmodel = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/Models/T5_Base")
hfmodel = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/QG_model")

In [32]:
hfmodel.cuda()

T5ForConditionalGeneration(
  (shared): Embedding(110080, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(110080, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo

In [28]:
def hf_run_model(input_string, **generator_args):
  generator_args = {
  "max_length": 256,
  "num_beams": 4,
  "length_penalty": 1.5,
  "no_repeat_ngram_size": 3,
  "early_stopping": True,
  }
  input_string = "generate fill-in the blank question: " + input_string + " </s>"
  input_ids = tokenizer.encode(input_string, return_tensors="pt").to('cuda')
  res = hfmodel.generate(input_ids, **generator_args)
  output = tokenizer.batch_decode(res, skip_special_tokens=True)
  output = [item.split("<sep>") for item in output]
  return output

In [23]:
text = " أبرز أسباب قيام حرب أكتوبر 1973 هو تمادى الجيش الإسرائيلي بعد نكسة 1967 خلال عهد الرئيس الراحل جمال عبد الناصر وإحتلال شبه جزيرة سيناء ووضع قواعد إسرائيلية عليها ،ومن الجدير بالذكر أن السبب الرئيسى وراء هزيمة 67 هو الخيانة التى تلقتها مصر فى ظهرها نتيجة حصولها على صفقة الأسلحة الفاسدة التى استخدمت فى الحرب"

In [33]:
hf_run_model(text)

[["'أبرز اسباب قيام حرب اكتوبر الوطنية هو تمادى الجيش..... بعد نكسة 1967 خلال عهد الرئيس الراحل جمال عبد الناصر وإحتلال شبه جزيرة سيناء ووضع قواعد..... عليها ومن الجدير بالذكر ان السبب الرئيسى وراء هزيمة 67 هو الخيانة التي تلقتها مصر في ظهرها نتيجة حصولها على صفقة الاسلحة فاسدة التي استخدمت في الحرب الحرب'"]]